In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [11]:
df = pd.read_csv("Cleaned_Apartments.csv")

data cleaning

In [12]:
df = df.drop(columns=['id'])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   city                  147 non-null    int64  
 1   type                  147 non-null    int64  
 2   squareMeters          147 non-null    float64
 3   rooms                 147 non-null    int64  
 4   floor                 147 non-null    int64  
 5   floorCount            147 non-null    int64  
 6   buildYear             147 non-null    int64  
 7   latitude              147 non-null    float64
 8   longitude             147 non-null    float64
 9   centreDistance        147 non-null    float64
 10  poiCount              147 non-null    int64  
 11  schoolDistance        147 non-null    float64
 12  clinicDistance        147 non-null    float64
 13  postOfficeDistance    147 non-null    float64
 14  kindergartenDistance  147 non-null    float64
 15  restaurantDistance    1

In [14]:
# Extracting sorted_prices list
df['sorted_prices'] = df['sorted_prices'].str.strip('[]').str.split(', ')

# Creating individual columns for prices
for i in range(len(df['sorted_prices'].iloc[0])):
    df[f'price_{i+1}'] = df['sorted_prices'].apply(lambda x: int(x[i]))

# Dropping the original sorted_prices column
df.drop(columns=['sorted_prices'], inplace=True)

df

,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,...,hasElevator,hasSecurity,hasStorageRoom,price_1,price_2,price_3,price_4,price_5,price_6,price_7
0,1,3,77.71,3,3,3,1925,53.433019,14.569775,1.22,...,0,0,1,550000,550000,550000,650000,599000,599000,599000
1,1,3,78.60,3,2,4,1929,53.432833,14.548187,1.35,...,0,0,1,440000,440000,440000,440000,440000,440000,440000
2,1,3,100.00,3,4,4,1930,53.436494,14.572026,1.63,...,0,0,1,799000,799000,739000,739000,739000,739000,739000
3,1,2,96.06,4,3,3,2009,53.465600,14.541539,4.87,...,0,0,0,768000,768000,799000,799000,799000,799000,799000
4,1,3,42.57,2,1,1,1938,53.495690,14.584651,8.19,...,0,0,1,389000,389000,379000,379000,379000,379000,379000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,14,1,78.90,3,2,2,2004,53.108380,17.980800,2.02,...,0,0,1,640000,640000,640000,640000,640000,640000,640000
143,14,3,39.00,2,1,4,1914,53.126000,18.007900,0.65,...,0,0,1,205000,205000,205000,205000,205000,205000,205000
144,14,2,37.30,2,10,10,1980,53.124984,18.049752,3.29,...,1,0,1,219000,219000,219000,219000,219000,219000,219000
145,14,3,95.00,4,1,3,1893,53.126000,18.007900,0.65,...,0,0,1,599000,599000,599000,599000,599000,599000,599000


In [15]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

# Split into input (X) and output (y) variables
X = scaled_data[:, :-1]  # All columns except the last one (price_7) as input
y = scaled_data[:, -1]    # Last column (price_7) as output

# Reshape input data to 3D tensor (samples, timesteps, features)
# Assuming you want to consider previous 6 prices to predict the next one (change it as necessary)
timesteps = 6
X_reshaped = np.array([X[i:timesteps+i] for i in range(len(X)-timesteps)])
y_reshaped = y[timesteps:]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_reshaped, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=1))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)


Epoch 1/50
3/3 [==============================] - 6s 673ms/step - loss: 0.0750 - val_loss: 0.0859
Epoch 2/50
3/3 [==============================] - 0s 25ms/step - loss: 0.0259 - val_loss: 0.0678
Epoch 3/50
3/3 [==============================] - 0s 32ms/step - loss: 0.0354 - val_loss: 0.0657
Epoch 4/50
3/3 [==============================] - 0s 26ms/step - loss: 0.0281 - val_loss: 0.0685
Epoch 5/50
3/3 [==============================] - 0s 34ms/step - loss: 0.0233 - val_loss: 0.0784
Epoch 6/50
3/3 [==============================] - 0s 43ms/step - loss: 0.0244 - val_loss: 0.0822
Epoch 7/50
3/3 [==============================] - 0s 28ms/step - loss: 0.0245 - val_loss: 0.0758
Epoch 8/50
3/3 [==============================] - 0s 27ms/step - loss: 0.0217 - val_loss: 0.0686
Epoch 9/50
3/3 [==============================] - 0s 28ms/step - loss: 0.0206 - val_loss: 0.0635
Epoch 10/50
3/3 [==============================] - 0s 27ms/step - loss: 0.0204 - val_loss: 0.0620
Epoch 11/50
3/3 [===========

In [16]:
model.save('my_lstm_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
df.drop(columns=['price_1', 'price_2', 'price_3', 'price_4', 'price_5', 'price_6'], inplace=True)
df

,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price_7
0,1,3,77.71,3,3,3,1925,53.433019,14.569775,1.22,...,0.430,0,0,0,0,0,0,0,1,599000
1,1,3,78.60,3,2,4,1929,53.432833,14.548187,1.35,...,0.089,0,0,1,0,0,0,0,1,440000
2,1,3,100.00,3,4,4,1930,53.436494,14.572026,1.63,...,0.191,0,0,1,1,0,0,0,1,739000
3,1,2,96.06,4,3,3,2009,53.465600,14.541539,4.87,...,0.366,0,0,1,0,1,0,0,0,799000
4,1,3,42.57,2,1,1,1938,53.495690,14.584651,8.19,...,0.033,0,0,1,1,0,0,0,1,379000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,14,1,78.90,3,2,2,2004,53.108380,17.980800,2.02,...,0.591,0,0,1,0,1,0,0,1,640000
143,14,3,39.00,2,1,4,1914,53.126000,18.007900,0.65,...,0.205,0,0,0,0,0,0,0,1,205000
144,14,2,37.30,2,10,10,1980,53.124984,18.049752,3.29,...,0.155,1,1,0,0,1,1,0,1,219000
145,14,3,95.00,4,1,3,1893,53.126000,18.007900,0.65,...,0.205,0,0,0,0,0,0,0,1,599000


In [19]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

# Split into input (X) and output (y) variables
X = scaled_data[:, :-1]  # All columns except the last one (price_7) as input
y = scaled_data[:, -1]    # Last column (price_7) as output

# Reshape input data to 3D tensor (samples, timesteps, features)
# Assuming you want to consider previous 6 prices to predict the next one (change it as necessary)
timesteps = 6
X_reshaped = np.array([X[i:timesteps+i] for i in range(len(X)-timesteps)])
y_reshaped = y[timesteps:]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_reshaped, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=1))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

Epoch 1/50
3/3 [==============================] - 5s 466ms/step - loss: 0.0633 - val_loss: 0.0795
Epoch 2/50
3/3 [==============================] - 0s 25ms/step - loss: 0.0299 - val_loss: 0.0711
Epoch 3/50
3/3 [==============================] - 0s 26ms/step - loss: 0.0361 - val_loss: 0.0697
Epoch 4/50
3/3 [==============================] - 0s 24ms/step - loss: 0.0259 - val_loss: 0.0774
Epoch 5/50
3/3 [==============================] - 0s 27ms/step - loss: 0.0235 - val_loss: 0.0889
Epoch 6/50
3/3 [==============================] - 0s 26ms/step - loss: 0.0252 - val_loss: 0.0909
Epoch 7/50
3/3 [==============================] - 0s 26ms/step - loss: 0.0242 - val_loss: 0.0833
Epoch 8/50
3/3 [==============================] - 0s 25ms/step - loss: 0.0211 - val_loss: 0.0749
Epoch 9/50
3/3 [==============================] - 0s 30ms/step - loss: 0.0206 - val_loss: 0.0695
Epoch 10/50
3/3 [==============================] - 0s 24ms/step - loss: 0.0208 - val_loss: 0.0689
Epoch 11/50
3/3 [===========

In [20]:
model.save('lstm_model_price7.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
